# CrysFML: Test against FullProf - 2
This is a comparison of CFML 0.0.3 from PyPI with FullProf.2k Version 7.30 - Mar2020-ILL JRC for macOS

In [ ]:
# Misc
import sys, os, re
import numpy as np
from io import StringIO
from pathlib import Path

# CrysFML
import CFML_api

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [ ]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

In [ ]:
def diffraction_pattern_xy(space_group, cell, atom_list, job_info):
    reflection_list = CFML_api.ReflectionList(cell, space_group, 
                                              True, 
                                              job_info)
    reflection_list.compute_structure_factors(space_group, 
                                              atom_list, 
                                              job_info)
    diffraction_pattern = CFML_api.DiffractionPattern(job_info,
                                                  reflection_list, 
                                                  cell.reciprocal_cell_vol)
    return diffraction_pattern.x, diffraction_pattern.ycalc

## Define sample model

### Create space group object

In [ ]:
space_group = CFML_api.SpaceGroup('P n m a')
#space_group.print_description()

### Create unit cell object

In [ ]:
lengths = np.asarray([8.485680,5.402480,6.964544], dtype='float32')
angles = np.asarray([90,90,90], dtype='float32')
unit_cell = CFML_api.Cell(lengths, angles)
#unit_cell.print_description()

### Create list of atoms

In [ ]:
cif_str = '''
loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_occupancy
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb  1.0  0.18797  0.25    0.16754  Uiso  0.0173
  S   S   1.0  0.06467  0.25    0.68300  Uiso  0.0115
  O1  O   1.0  0.90705  0.25    0.59672  Uiso  0.0070
  O2  O   1.0  0.18641  0.25    0.54269  Uiso  0.0127
  O3  O   1.0  0.08025  0.02961 0.81208  Uiso  0.0135
'''
atom_list = CFML_api.AtomList(cif_str.split('\n'))
atom_list.set_mult_occ_cif(space_group)
atom_list.print_description()

## Load simulated data

### Load simulated data

In [ ]:
file_path = '../datasets/fullprof_simulations/pbsox_2_data.cif'
from_str = '_pd_proc_intensity_bkg_calc'
to_str = '_pd_proc_number_of_points'

file_content = Path(file_path).read_text()
match = re.search(f'{from_str}(.*){to_str}', file_content, re.DOTALL)

data_str = match.group(0)
data_str = data_str.replace(from_str, '').replace(to_str, '')
data_str = data_str.replace('(','  ').replace(')','  ')

_, x_calc_fp, _, _, y_calc_fp, _ = np.genfromtxt(StringIO(data_str), unpack=True)

### Visualize simulated data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_calc_fp, y_calc_fp, legend_label='Icalc (FullProf)', color='darkgray', line_width=2)
show(fig)

## Analyse data

### Create job definition object

In [ ]:
cfl_str = '''
Title PbSO4
Patt_1 XRAY_2THE  1.540560    1.540560    1.00      0.0        135.0
'''
job_info = CFML_api.JobInfo(cfl_str.split('\n'))
job_info.print_description()

### Modify job definition object

In [ ]:
job_info.pattern_type = "XRAY_2THE"
job_info.range_2theta = (10.0, 154.0)
job_info.theta_step = 0.025
#job_info.lambdas = (1.5405, 1.5444)
#job_info.lambda_ratio = 0.5
job_info.u_resolution = 0.031112
job_info.v_resolution = -0.052102
job_info.w_resolution = 0.032035
job_info.x_resolution = 0.0
job_info.y_resolution = 0.0
job_info.print_description()

### Calculate and plot diffraction pattern

In [ ]:
x_calc_cfml, y_calc_cfml = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_calc_cfml, y_calc_cfml, legend_label='Icalc (CrysFML)', color='orangered', line_width=1)
show(fig)

### Compare observed and calculated data

In [ ]:
scale = 0.25
background = 100
zero_shift = -0.0

In [ ]:
x_calc_cfml, y_calc_cfml = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
x_calc_cfml = x_calc_cfml + zero_shift
y_calc_cfml = y_calc_cfml * scale + background

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_calc_fp, y_calc_fp, legend_label='Icalc (FP)', color='darkgray', line_width=2)
fig.line(x_calc_cfml, y_calc_cfml, legend_label='Icalc (CFML)', color='orangered', line_width=1)
show(fig)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(20,30))
fig.line(x_calc_fp, y_calc_fp, legend_label='Icalc (FP)', color='darkgray', line_width=2)
fig.line(x_calc_cfml, y_calc_cfml, legend_label='Icalc (CFML)', color='orangered', line_width=1)
show(fig)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(43,46.5))
fig.line(x_calc_fp, y_calc_fp, legend_label='Icalc (FP)', color='darkgray', line_width=2)
fig.line(x_calc_cfml, y_calc_cfml, legend_label='Icalc (CFML)', color='orangered', line_width=1)
show(fig)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, x_range=(61,65))
fig.line(x_calc_fp, y_calc_fp, legend_label='Icalc (FP)', color='darkgray', line_width=2)
fig.line(x_calc_cfml, y_calc_cfml, legend_label='Icalc (CFML)', color='orangered', line_width=1)
show(fig)